In [1]:
import os
import requests
from blaze import *
import pandas as pd

C:\Miniconda3\lib\site-packages\flask\exthook.py:71: ExtDeprecationWarning: Importing flask.ext.cors is deprecated, use flask_cors instead.
  .format(x=modname), ExtDeprecationWarning


In [12]:
data_dir = "D:/Users/cmarciniak/Documents/macmap/data/comtrade"
file = "type-C_r-ALL_ps-1995_freq-A_px-HS_pub-20060109_fmt-csv_ex-20151117.csv"
comtrade95 = data(os.path.join(data_dir,file))


In [67]:
chunk_size = 100000
reader = pd.read_table(comtrade95,sep=",",chunksize=chunk_size,dtype='object')

chunk = reader.get_chunk(chunk_size*10)


Products listed at the 6,4, and 2 digit level

In [73]:
# remove TOTAL Rows
subset = chunk[chunk["Commodity Code"]!="TOTAL"]
subset["Commodity Code"].str.len().value_counts()

6    655943
4    284372
2     57957
Name: Commodity Code, dtype: int64

In [124]:
reader = pd.read_table(os.path.join(data_dir,"comtradeHS2014.zip"),sep=',',chunksize=chunk_size,dtype='object')
comtrade14 = reader.get_chunk(chunk_size)
comtrade14 = comtrade14.rename(columns= lambda x : x.replace(" ","_"))
subset14 = comtrade14[comtrade14["Commodity_Code"]!="TOTAL"]
subset14['Commodity_Code'].str.len().value_counts()

6    62945
4    27126
2     9660
Name: Commodity_Code, dtype: int64

In [59]:
url = "http://comtrade.un.org/api//refs/da/bulk?ps=2013" # gives download uris for bulk requests for 2013
res = requests.get(url)

In [28]:
# commodities, annual, 2013, rep=24, H0 classification
# ex http://comtrade.un.org/api/get/bulk/C/A/2013/24/H0

len(res.json())

1878

In [43]:
# Examine the same dataset with two different classifications
h1_url = "http://comtrade.un.org/api/get/bulk/C/A/2014/28/H1"
h2_url = "http://comtrade.un.org/api/get/bulk/C/A/2014/28/H2"
# access appears to be blocked by COMTRADE
#res = requests.get(h1_url) 

In [ ]:
# Data Extraction API
da_url = "http://comtrade.un.org/api//refs/da/view?parameters"
res = requests.get(da_url)



In [37]:
available_data = pd.io.json.json_normalize(res.json())

In [48]:
len(available_data[available_data.type=="COMMODITIES"])

45338

In [49]:
data = "http://comtrade.un.org/api/refs/da/view?type=C&freq=M&ps=201607&px=HS"

In [45]:
# Iterate through bulk download csv of 2013
chunksize = 10 ** 6

trade = pd.read_table(file,sep=",",chunksize=chunksize)
for chunk in trade:
    print(chunk)

       ﻿Classification  Year  Period  Period Desc.  Aggregate Level  \
0                   H2  2013    2013          2013                2   
1                   H2  2013    2013          2013                2   
2                   H2  2013    2013          2013                2   
3                   H2  2013    2013          2013                2   
4                   H2  2013    2013          2013                2   
5                   H2  2013    2013          2013                2   
6                   H2  2013    2013          2013                2   
7                   H2  2013    2013          2013                2   
8                   H2  2013    2013          2013                2   
9                   H2  2013    2013          2013                2   
10                  H2  2013    2013          2013                2   
11                  H2  2013    2013          2013                2   
12                  H2  2013    2013          2013                2   
13    

KeyboardInterrupt: 

In [120]:
comtrade14['Qty_Unit'].value_counts()

Weight in kilograms                                 63992
Number of items                                     18270
No Quantity                                         15939
Number of pairs                                       641
Area in square metres                                 567
Volume in litres                                      410
Volume in cubic meters                                162
Thousands of items                                     13
Electrical energy in thousands of kilowatt-hours        4
Number of packages                                      2
Name: Qty Unit, dtype: int64

In [96]:
def remove_total(dataframe):
    return dataframe[dataframe['Commodity_Code'] is not "TOTAL"]


In [105]:
# Two-digit reporters
subset14[subset14['Commodity_Code'].str.len()==2]['Reporter'].value_counts()

Albania    4992
Algeria    4668
Name: Reporter, dtype: int64

In [106]:
subset14[subset14['Commodity_Code'].str.len()==4]['Reporter'].value_counts()

Algeria    21729
Albania     5397
Name: Reporter, dtype: int64

In [111]:
subset14.Reporter.unique()

array(['Albania', 'Algeria'], dtype=object)

In [158]:

#ds = dshape("var * {Commodity_Code: string, Qty: int}")
cmtrd14 = data(os.path.join(data_dir,file),dtype='object')
cmtrd14.dshape  # Get dshape from data and correct Commodity Code to string
ds = dshape("""var * {
  '\ufeffClassification': ?string,
  Year: int64,
  Period: int64,
  'Period Desc.': int64,
  'Aggregate Level': int64,
  'Is Leaf Code': int64,
  'Trade Flow Code': int64,
  'Trade Flow': ?string,
  'Reporter Code': int64,
  Reporter: ?string,
  'Reporter ISO': ?string,
  'Partner Code': int64,
  Partner: ?string,
  'Partner ISO': ?string,
  'Commodity Code': string,
  Commodity: ?string,
  'Qty Unit Code': int64,
  'Qty Unit': ?string,
  Qty: ?float64,
  'Netweight (kg)': ?float64,
  'Trade Value (US$)': int64,
  Flag: int64
  }""")
cmtrd14 = data(cmtrd14, dshape=ds)

In [192]:
subset14 = cmtrd14[cmtrd14['Commodity Code']!="TOTAL"]

In [193]:
average_qty = compute(by( subset14['Commodity Code'],average=subset14['Qty'].mean()))

In [183]:
count = by( cmtrd14[['Reporter','Trade Flow','Commodity Code']], count= cmtrd14['Qty'].count() )
srt = by(cmtrd14[['Reporter','Trade Flow','Commodity Code']], sort= cmtrd14['Qty'].sort())


In [233]:
srt = subset14.sort(['Reporter','Trade Flow','Commodity Code'])
nelements = compute(by(srt[['Reporter','Trade Flow','Commodity Code']],n=srt.Qty.count()))

In [230]:
nelements

,Reporter,Trade Flow,Commodity Code,n
0,Algeria,Export,03,8
1,Algeria,Export,0301,4
2,Algeria,Export,030192,4
3,Algeria,Export,0302,8
4,Algeria,Export,030219,3
5,Algeria,Export,030223,2
6,Algeria,Export,030229,3
7,Algeria,Export,030269,6
8,Algeria,Export,0303,2
9,Algeria,Export,030339,2


In [200]:
med = compute(subset14.Qty.map(np.median,'float64'))

C:\Miniconda3\lib\site-packages\numpy\lib\function_base.py:3569: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


In [197]:
compute(cmtrd14.Flag.count_values())

,Flag,count
0,0,12678998


In [173]:
cmtrd14[cmtrd14['Commodity Code']!="TOTAL"]

,﻿Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,0,World,WLD,01,Live animals,1,No Quantity,NaN,NaN,15841726,0
1,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,58,Belgium-Luxembourg,BEL,01,Live animals,1,No Quantity,NaN,NaN,384448,0
2,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,251,France,FRA,01,Live animals,1,No Quantity,NaN,NaN,12232970,0
3,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,276,Germany,DEU,01,Live animals,1,No Quantity,NaN,NaN,645205,0
4,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,348,Hungary,HUN,01,Live animals,1,No Quantity,NaN,NaN,709825,0
5,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,381,Italy,ITA,01,Live animals,1,No Quantity,NaN,NaN,1384977,0
6,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,528,Netherlands,NLD,01,Live animals,1,No Quantity,NaN,NaN,301946,0
7,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,562,Niger,NER,01,Live animals,1,No Quantity,NaN,NaN,2522,0
8,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,643,Russian Federation,RUS,01,Live animals,1,No Quantity,NaN,NaN,5482,0
9,H0,1995,1995,1995,2,0,1,Import,12,Algeria,DZA,724,Spain,ESP,01,Live animals,1,No Quantity,NaN,NaN,174035,0
